In [ ]:
# Exemplo EDA sobre o dataset de datos de coches
# https://www.datafied.world/eda-on-mpg-data-using-seaborn-192

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 'mpg' é un coñecido dataset con datos de automóbiles que se encontra como exemplo na libraría 'seaborn'
# Tamén dispoñible no UCI: Machine Learning Repository
# https://archive.ics.uci.edu/ml/datasets/auto+mpg

# O repositorio proporciona a seguinte información:
# Attribute Information:

# 1. mpg: continuous
# 2. cylinders: multi-valued discrete
# 3. displacement: continuous
# 4. horsepower: continuous
# 5. weight: continuous
# 6. acceleration: continuous
# 7. model year: multi-valued discrete
# 8. origin: multi-valued discrete
# 9. car name: string (unique for each instance)

In [ ]:
df = sns.load_dataset('mpg')

In [ ]:
# Fases do EDA
# 1. Preprocesar os datos
# 2. EDA sobre atributos categóricos: analizar distribución e relacións con outros categóricos
# 3. EDA sobre atributos numéricos: analizar distribució e relacións con outros numéricos
# 4. Análise da relación entre categóricos e numéricos

In [ ]:
# PREPROCESADO DOS DATOS

In [ ]:
# Unha ollada aos datos

df.head()

In [ ]:
# Tamaño do dataset? Filas? Atributos?
df.shape

In [ ]:
# Listar todas as columnas
columns = list(df.columns)
columns

In [ ]:
# Consultar os tipos de datos, se hai valores nulos..
df.info()

In [ ]:
# Creamos dúas listas diferentes:
# - nomes das columnas categóricas
# - nomes das columnas numéricas 
# Isto facilitará o tratamento das diferentes variables por separado

cats = list(df.select_dtypes(include=['object']).columns)
nums = list(df.select_dtypes(exclude=['object']).columns)
print(f'Variables categóricas: {cats}')
print(f'Variables numéricas: {nums}')

In [ ]:
# Consultamos o número de valores diferentes en cada columna
df.nunique(axis=0)

In [ ]:
# cylinders e model_year teñen moi pouca variabilidade -> ten sentido facelos categóricos
# Movémolos á lista de variables categóricas

In [ ]:
cats.extend(['cylinders','model_year'])
nums.remove('cylinders')
nums.remove('model_year')
print(f'Variables categóricas: {cats}')
print(f'Variables numéricas: {nums}')

In [ ]:
# búsqueda de nans - TRATAMENTO DE MISSING VALUES
df.isna().sum()
# df.isnull().sum() <- isnull() é un alias de isna()

In [ ]:
# mostrar as liñas que teñen nan
df[df.horsepower.isnull()]

In [ ]:
# mais xeralmente
df[df.isnull().any(axis=1)]

In [ ]:
# proporción de nans
6 / len(df)

In [ ]:
# A porcentaxe é baixa (un 1,5%), logo podemos machacar esas liñas
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.shape

In [ ]:

# df = df[~df.isnull().any(axis=1)]
# df.reset_index(inplace=True)
# df.drop('index', inplace=True, axis=1)

In [ ]:
# TRATAMENTO DE DUPLICADOS

In [ ]:
# Buscar filas duplicadas
print(f'Total de filas duplicadas: {df.duplicated().sum()}')

In [ ]:
# No caso de que houbese duplicados (que non hai), eliminamos
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# df = df[~df.duplicated()]
# df.shape

In [ ]:
# Podemos agrupar as variables por tipo por comodidade (recolocar as columnas)
# bloque categóticas vs bloque numéricas
df = pd.concat((df[cats], df[nums]), axis=1)
df.head()

In [ ]:
num_rows, num_cols = df.shape

In [ ]:
# Pode ser unha boa idea gardar os datos unha vez limpos e preprocesados
df.to_csv('mpg_cleaned.csv',index=False)

In [ ]:
# ANÁLISE DOS VALORES CATEGÓRICOS

# Despois da limpeza dos datos procedemos á análise das variables categóricas

In [ ]:
df = pd.read_csv('mpg_cleaned.csv')

In [ ]:
print(f'categorical variables: {cats}')

In [ ]:
# Creamos un novo dataframe só coas variables categóricas

#df_cat = df.loc[:, 'origin':'model_year']
df_cat= df[cats]
df_cat.head()

In [ ]:
# limpeza de columnas tipo string, eliminando espazos extra (por precaución)
# Ollo a: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
for col in ['origin', 'name']:
    df_cat[col] = df_cat[col].apply(lambda x: ' '.join(x.split()))

In [ ]:
# Pode ser útil crear niveis categóricos para algunhas variables (p.ex: mpg)
# Axudará á hora de crear algúns gráficos e ademais facilita a comprensión
# Engadimos unha nova variable categórica

In [ ]:
df_cat['mpg_level'] = df['mpg'].apply(lambda x: 'low' if x<17 else 'high' if x>29 else 'medium')
cats.append('mpg_level')
print(f'Variables categóricas:  {cats}')

In [ ]:
# Botar unha ollada aos valores das categorías
print(f"Categorías en origin: {pd.unique(df_cat['origin'])}")
print(f"Categorías en cylinders: {pd.unique(df_cat['cylinders'])}")
print(f"Categorías en model_year: {pd.unique(df_cat['model_year'])}")

In [ ]:
# Análise da distribución 
#
# Queremos ver como se distribúen os datos e sacar algunha información das gráficas

In [ ]:
# Countplot pode ser unha boa opción para ver como se distribúen os datos nas variables categóricas
sns.set_theme(style='darkgrid')

# Distribución de "orixe"
sns.countplot(data=df_cat,x='origin')

In [ ]:
# A maioría dos datos son de coches estadounidenses

In [ ]:
# Distribución de "cilindros"
sns.countplot(data=df_cat,x='cylinders')

In [ ]:
# O mais común son coches con motores de 4 ciclindros. As versións de 3 e 5 son anecdóticas.

In [ ]:
# Distribución dos datos segundo "model_year"
sns.countplot(data=df_cat,x='model_year')

In [ ]:
# Distribución de "mpg_level"
sns.countplot(data=df_cat,x='mpg_level')

In [ ]:
# Pode ter sentido ordenar as barras en función de importancia

In [ ]:
plt.figure(figsize=(15,5))
ax = plt.subplot(1,2,1)
ax.set_title('Orde temporal')
sns.countplot(data=df_cat,x='model_year')
ax = plt.subplot(1,2,2)
ax.set_title('Orde de relevancia')
sns.countplot(data=df_cat,x='model_year',order=df_cat.model_year.value_counts().index)

In [ ]:
# Podemos agrupar as gráficas

In [ ]:
fig = plt.figure(1, (14, 8))

for i,cat in enumerate(df_cat.drop(['name'], axis=1).columns):
    ax = plt.subplot(2,2,i+1)
    sns.countplot(data=df_cat,x=cat, order=df_cat[cat].value_counts().index)
    ax.set_xlabel(None)
    ax.set_title(f'Distribution of {cat}')
    plt.tight_layout()

plt.show()

In [ ]:
# Cálculo da proporción das clases dominantes en relación co resto da súa categoría
for i,cat in enumerate(df_cat.drop(['name'], axis=1).columns):
    val_counts = df_cat[cat].value_counts()
    dominant_frac = val_counts.iloc[0] / num_rows
    print(f'`{val_counts.index[0]}` contribúe por si sóa nun {round(dominant_frac * 100, 2)}% de {cat}')

In [ ]:
# Funcionamento de value_counts
# df_cat.cylinders.value_counts()

In [ ]:
# NOVA INFORMACIÓN EXTRAÍDA DA ANÁLISE  -- Insights

# - Orixe desequilibrado en favor de usa, maior que a suma dos competidores
# - Cilindros desequilibrado en favor de 4
# - mpg_level (que fixemos categórica) desequilibrio en favor de medium (que é un rango maior)
# - model_year -> equilibrado

In [ ]:
# analizar os nomes dos modelos 'name'
print(f'Número de categorías diferentes en  `name`: {df_cat.name.nunique()}')
print(f"\nListaxe de categorías en `name`:\n\n {df_cat.name.unique()}")

In [ ]:
# Os nomes inclúen a marca!

# Podemos extraer a marca/compañía de `name` , coa intención de crear unha nova categoría 
df_cat['car_company'] = df_cat['name'].apply(lambda x: x.split()[0])

# Eliminamos a Compañía e deixamos só o modelo
df_cat['car_name'] = df_cat['name'].apply(lambda x: ' '.join(x.split()[1:]))
df_cat.drop('name', axis=1, inplace=True)

cats.extend(['car_company', 'car_name'])
cats.remove('name')

print(f'Variables categóricas:  {cats}')
df_cat.head()

In [ ]:
# Investigamos as categorías en `car_company`
print(f'Número de categorías en `car_company`: {df_cat.car_company.nunique()}')
print(f"\nListaxe de categorías en `car_company`:\n\n {df_cat.car_company.unique()}")

In [ ]:
# Agora xa podemos analizar a distribución dos datos en función da marca/compañía
fig = plt.figure(1, (18, 4))

ax1 = plt.subplot(1,1,1)
sns.countplot(data=df_cat,x='car_company', order=df_cat['car_company'].value_counts().index)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=75)

plt.show()

In [ ]:
df_cat.car_company.value_counts()[:2]

In [ ]:
# NOVA INFORMACIÓN EXTRAÍDA DA ANÁLISE  -- Insights

# - car_name ten demasiadas categorías, practicamente unha por liña, polo que non nos daría ningunha información útil
# - creamos unha nova variable 'car_company' para as marcas dos coches, reducindo en gran medida o número de categorías
# - a distribución de car_company non é uniforme e poucas compañías abarcan a maior parte dos modelos de coche

In [ ]:
# CONCLUSIÓNS

# * Todos os atributos categóricos (excepto model_year) están moi desbalanceados e lonxe da distribución uniforme.
# * En todos os datos concéntranse en poucas categorías